# LAB 3: How to setup a project from Scratch

In [1]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [2]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b RavdnessClass https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 296 (delta 143), reused 120 (delta 65), pack-reused 94 (from 3)
Receiving objects: 100% (296/296), 111.54 KiB | 1.66 MiB/s, done.
Resolving deltas: 100% (179/179), done.


In [3]:
!ls

sample_data  speech-emotion-recognition-25


In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [8]:
# ============================================================================
# SMART DATASET DOWNLOAD - Scarica solo se non presente
# ============================================================================

from pathlib import Path

ravdess_path = Path('./ravdess')
iemocap_path = Path('./iemocap')

# Verifica se i dataset esistono già
ravdess_exists = ravdess_path.exists() and any(ravdess_path.iterdir())
iemocap_exists = iemocap_path.exists() and any(iemocap_path.iterdir())

if ravdess_exists and iemocap_exists:
    print("✅ Dataset già presenti, skip download")
    print(f"   📁 RAVDESS: {ravdess_path}")
    print(f"   📁 IEMOCAP: {iemocap_path}")
else:
    print("⬇️  Download dataset in corso...")
    !python utils/download_dataset.py

⬇️  Download dataset in corso...
--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/ravdess-emotional-speech-audio
Copia dei file nella cartella di lavoro: ./ravdess...
RAVDESS pronto in: ./ravdess
Numero totale di file copiati: 2880

RIEPILOGO DOWNLOAD
RAVDESS: ✅ Successo

🎉 Tutti i dataset sono stati scaricati con successo!


In [9]:
# ============================================================================
# ESPLORAZIONE STRUTTURA DATASET
# ============================================================================

from pathlib import Path
from collections import defaultdict

def analyze_dataset_structure(dataset_path, dataset_name):
    """Analizza e mostra la struttura di un dataset"""
    print("=" * 80)
    print(f"📁 STRUTTURA DATASET: {dataset_name.upper()}")
    print("=" * 80)
    
    path = Path(dataset_path)
    
    if not path.exists():
        print(f"❌ Directory non trovata: {dataset_path}")
        return
    
    # 1. STATISTICHE GENERALI
    print("\n📊 STATISTICHE GENERALI:")
    print("-" * 80)
    
    total_files = 0
    total_size = 0
    file_extensions = defaultdict(int)
    
    for item in path.rglob('*'):
        if item.is_file():
            total_files += 1
            total_size += item.stat().st_size
            ext = item.suffix.lower() or '[no extension]'
            file_extensions[ext] += 1
    
    print(f"  • Totale file: {total_files}")
    print(f"  • Dimensione totale: {total_size / (1024**3):.2f} GB")
    print(f"  • Tipi di file:")
    for ext, count in sorted(file_extensions.items(), key=lambda x: x[1], reverse=True):
        print(f"      {ext}: {count} file(s)")
    
    # 2. STRUTTURA DIRECTORY (primi 3 livelli)
    print("\n📂 STRUTTURA DIRECTORY (primi 3 livelli):")
    print("-" * 80)
    
    def print_tree(directory, prefix="", max_depth=3, current_depth=0):
        if current_depth >= max_depth:
            return
        
        try:
            items = sorted(directory.iterdir(), key=lambda x: (not x.is_dir(), x.name))
            dirs = [item for item in items if item.is_dir()]
            files = [item for item in items if item.is_file()]
            
            # Mostra prime 3 directory
            for i, item in enumerate(dirs[:3]):
                is_last = (i == len(dirs[:3]) - 1) and len(files) == 0
                print(f"{prefix}{'└── ' if is_last else '├── '}📁 {item.name}/")
                
                extension = "    " if is_last else "│   "
                print_tree(item, prefix + extension, max_depth, current_depth + 1)
            
            if len(dirs) > 3:
                print(f"{prefix}└── ... altre {len(dirs) - 3} cartelle")
            
            # Mostra primi 3 file
            if files and current_depth < max_depth - 1:
                for i, item in enumerate(files[:3]):
                    is_last = i == len(files[:3]) - 1
                    size_mb = item.stat().st_size / (1024**2)
                    print(f"{prefix}{'└── ' if is_last else '├── '}📄 {item.name} ({size_mb:.2f} MB)")
                
                if len(files) > 3:
                    print(f"{prefix}└── ... altri {len(files) - 3} file")
        
        except PermissionError:
            print(f"{prefix}❌ Accesso negato")
    
    print_tree(path)
    
    # 3. ESEMPIO FILE PATHS (primi 5)
    print("\n📝 ESEMPIO PERCORSI FILE (primi 5):")
    print("-" * 80)
    
    all_files = list(path.rglob('*'))
    file_list = [f for f in all_files if f.is_file()][:5]
    
    for f in file_list:
        relative_path = f.relative_to(path)
        size_mb = f.stat().st_size / (1024**2)
        print(f"  • {relative_path}")
        print(f"    Dimensione: {size_mb:.2f} MB")
    
    # 4. DISTRIBUZIONE PER LIVELLO
    print("\n📊 DISTRIBUZIONE FILE PER LIVELLO:")
    print("-" * 80)
    
    level_distribution = defaultdict(int)
    for item in path.rglob('*'):
        if item.is_file():
            depth = len(item.relative_to(path).parts) - 1
            level_distribution[depth] += 1
    
    for level in sorted(level_distribution.keys()):
        print(f"  Livello {level}: {level_distribution[level]} file(s)")
    
    print("\n" + "=" * 80 + "\n")


# ============================================================================
# ANALISI RAVDESS
# ============================================================================

print("\n" + "🎵" * 40)
print("ANALISI DATASET RAVDESS")
print("🎵" * 40 + "\n")

analyze_dataset_structure("./ravdess", "RAVDESS")


# ============================================================================
# ANALISI IEMOCAP
# ============================================================================

print("\n" + "🎙️" * 40)
print("ANALISI DATASET IEMOCAP")
print("🎙️" * 40 + "\n")

analyze_dataset_structure("./iemocap", "IEMOCAP")


🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵
ANALISI DATASET RAVDESS
🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵🎵

📁 STRUTTURA DATASET: RAVDESS

📊 STATISTICHE GENERALI:
--------------------------------------------------------------------------------
  • Totale file: 2880
  • Dimensione totale: 1.10 GB
  • Tipi di file:
      .wav: 2880 file(s)

📂 STRUTTURA DIRECTORY (primi 3 livelli):
--------------------------------------------------------------------------------
├── 📁 Actor_01/
│   ├── 📄 03-01-01-01-01-01-01.wav (0.36 MB)
│   ├── 📄 03-01-01-01-01-02-01.wav (0.36 MB)
│   └── 📄 03-01-01-01-02-01-01.wav (0.36 MB)
│   └── ... altri 57 file
├── 📁 Actor_02/
│   ├── 📄 03-01-01-01-01-01-02.wav (0.39 MB)
│   ├── 📄 03-01-01-01-01-02-02.wav (0.39 MB)
│   └── 📄 03-01-01-01-02-01-02.wav (0.40 MB)
│   └── ... altri 57 file
└── 📁 Actor_03/
    ├── 📄 03-01-01-01-01-01-03.wav (0.36 MB)
    ├── 📄 03-01-01-01-01-02-03.wav (0.37 MB)
    └── 📄 03-01-01-01-02-01-03.wav (0.37 MB)
    └── ... altri 57 file
└── ...

# Step 3.5: Improvised Vocal Recording

Model IEMOCAP dataset to extract only improvised audio. 

In [ ]:
## insert code here 

In [ ]:
# Create Data Loaders

# Step 3.6: Print Dataset Stats

In [13]:
from utils.get_statistics import print_dataset_stats

# 1. Creazione Dataset (Silenziosa)
train_ds = CustomRAVDESSDataset('./ravdess', split='train')
val_ds = CustomRAVDESSDataset('./ravdess', split='validation')

# 2. Analisi (Solo quando vuoi tu)
print_dataset_stats(train_ds, "Training Set")
print_dataset_stats(val_ds, "Validation Set")


📊 ANALISI TRAINING SET
🔹 Samples Totali: 1440
🔹 Attori (20): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
   - Maschi:  10
   - Femmine: 10

🎭 Distribuzione Emozioni:
   - Angry     :  320 ( 22.2%) ████
   - Happy     :  320 ( 22.2%) ████
   - Neutral   :  480 ( 33.3%) ██████
   - Sad       :  320 ( 22.2%) ████
----------------------------------------

📊 ANALISI VALIDATION SET
🔹 Samples Totali: 144
🔹 Attori (2): [21, 22]
   - Maschi:  1
   - Femmine: 1

🎭 Distribuzione Emozioni:
   - Angry     :   32 ( 22.2%) ████
   - Happy     :   32 ( 22.2%) ████
   - Neutral   :   48 ( 33.3%) ██████
   - Sad       :   32 ( 22.2%) ████
----------------------------------------


# Step 3.7: Create DataLoader

Create DataLoader for both Datasets

In [14]:
from dataset.custom_ravdess_dataset import CustomRAVDESSDataset

train_dataset = CustomRAVDESSDataset(dataset_root='./ravdess', split='train')
val_dataset = CustomRAVDESSDataset(dataset_root='./ravdess', split='validation')
test_dataset = CustomRAVDESSDataset(dataset_root='./ravdess', split='test')


from utils.create_dataloaders import create_dataloaders

train_loader, val_loader, test_loader = create_dataloaders(
    train_dataset, val_dataset, test_dataset,
    batch_size=32, device=device, dataset_name='RAVDESS'
)

📦 CREAZIONE DATALOADERS - RAVDESS

✅ DataLoaders creati:
   Train: 45 batches (1440 samples)
   Val: 5 batches (144 samples)
   Test: 5 batches (144 samples)


# Step 4: Train your model and visualize training

In [ ]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [ ]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [ ]:
#!python eval.py